<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/poprawa_danych/colabs/Zmiana_typu_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 200 kB 37.5 MB/s 
     |████████████████████████████████| 199 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=5d10bf790140e4b24b1b62ef2a7fff405d6e1425810a2d6320b3a3d52851d26e
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-08 14:32:08--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-08 14:32:42--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 52.84.106.120, 52.84.106.38, 52.84.106.125, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|52.84.106.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236081246 (225M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.14M  15.6MB/s    in 14s     

2022-11-08 14:32:57 (16.3 MB/s) - ‘name.basics.tsv.gz’ saved [236081246/236081246]

--2022-11-08 14:32:57--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 52.84.106.120, 52.84.106.38, 52.84.106.125, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|52.84.106.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286852961 (274M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 273.56M 

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basics.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basics=spark.read.option("header","true").option("delimiter", "\t").csv('title.basics.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [7]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basics.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0053137,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0037382,tt01170...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00573...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

#Dostosowanie typu danych

##Sprawdźmy aktualne typy danych

In [8]:
df_name_basics.printSchema()
df_title_akas.printSchema()
df_title_basics.printSchema()
df_title_crew.printSchema()
df_title_episode.printSchema()
df_title_principals.printSchema()
df_title_ratings.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)

root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: string (nullable = true)

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- tconst: string (nullabl

wszystkie dane domyślnie są typu string

##Zmiana typów danych

Typy danych zmienimy zgodnie z opisem danych https://www.imdb.com/interfaces/ 

In [9]:
from pyspark.sql.types import StringType, ArrayType, IntegerType, BooleanType, DoubleType, DateType

###zmiana string na integer

Typu integer powinny być: z tabeli df_title_akas dane z kolumny ordering; z tabeli df_title_principals dane z kolumny ordering, z tabeli df_title_episode dane z kolumn seasonNumber i episodeNumber, z tabeli df_title_ratings dane z kolumny numVotes, z df_title_basics kolumna runtimeMinutes. Mamy także dane w formacie YYYY, określające rok, je również zmienimy na integer: z tabeli df_title_basics kolumny startYear, endYear; z tabeli df_name_basics kolumny birthYear, deathYear.

In [45]:
df_title_akas = df_title_akas.withColumn("ordering", df_title_akas["ordering"].cast(IntegerType()))
df_title_principals = df_title_principals.withColumn("ordering", df_title_principals["ordering"].cast(IntegerType()))
df_title_ratings = df_title_ratings.withColumn("numVotes", df_title_ratings["numVotes"].cast(IntegerType()))
df_title_episode = df_title_episode.withColumn("seasonNumber", df_title_episode["seasonNumber"].cast(IntegerType())).withColumn("episodeNumber", df_title_episode["episodeNumber"].cast(IntegerType()))
df_title_basics = df_title_basics.withColumn("startYear", df_title_basics["startYear"].cast(IntegerType())).withColumn("endYear", df_title_basics["endYear"].cast(IntegerType())).withColumn("runtimeMinutes", df_title_basics["runtimeMinutes"].cast(IntegerType()))
df_name_basics = df_name_basics.withColumn("birthYear", df_name_basics["birthYear"].cast(IntegerType())).withColumn("deathYear", df_name_basics["deathYear"].cast(IntegerType()))

###zamiana string na boolean

Zmienimy z tabeli df_title_akas kolumnę isOriginalTitle oraz
z tabeli df_title_basics kolumnę isAdult

In [16]:
df_title_akas = df_title_akas.withColumn("isOriginalTitle", df_title_akas["isOriginalTitle"].cast(BooleanType()))
df_title_basics = df_title_basics.withColumn("isAdult", df_title_basics["isAdult"].cast(BooleanType()))

###zamiana string na Double

df_title_ratings averageRating 

In [19]:
df_title_ratings = df_title_ratings.withColumn("averageRating", df_title_ratings["averageRating"].cast(DoubleType()))

###zamiana string na array(string)

In [22]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import split, col

In [21]:
#df_title_akas = df_title_akas.withColumn("types", df_title_akas["types"].cast(ArrayType()))

Zmienimy następujące kolumny: z tabeli df_title_akas kolumny attributes, types;
z tabeli df_title_basics kolumna genres;
z tabeli df_name_basics kolumny primaryProfession, knownForTitles;
z tabeli df_title_crew kolumny directors, writers.


####Stworzenie nowych, odrębnych kolumn typu array, z kolumn typu string określonych wyżej

In [36]:
df_title_akas_types= df_title_akas.select(split(col("types"),",").alias("types"))
df_title_akas_attributes= df_title_akas.select(split(col("attributes"),",").alias("attributes"))
df_name_basics_primaryProfession= df_name_basics.select(split(col("primaryProfession"),",").alias("primaryProfession"))
df_name_basics_knownForTitles= df_name_basics.select(split(col("knownForTitles"),",").alias("knownForTitles"))
df_title_basics_genres=df_title_basics.select(split(col("genres"),",").alias("genres"))
df_title_crew_directors=df_title_crew.select(split(col("directors"),",").alias("directors"))
df_title_crew_writers=df_title_crew.select(split(col("writers"),",").alias("writers"))

wyświetlmy ich typy danych

In [37]:
df_title_akas_types.printSchema()
df_title_akas_attributes.printSchema()
df_name_basics_primaryProfession.printSchema()
df_name_basics_knownForTitles.printSchema()
df_title_basics_genres.printSchema()
df_title_crew_directors.printSchema()
df_title_crew_writers.printSchema()

root
 |-- types: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- attributes: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- primaryProfession: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- knownForTitles: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- directors: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- writers: array (nullable = true)
 |    |-- element: string (containsNull = false)



####Zamieńmy pierwotne kolumny na nowe

#####usuńmy pierwotne kolumny z niewłaściwym typem danych

In [39]:
df_title_akas= df_title_akas.drop("types")
df_title_akas= df_title_akas.drop("attributes")
df_name_basics= df_name_basics.drop("primaryProfession")
df_name_basics= df_name_basics.drop("knownForTitles")
df_title_basics=df_title_basics.drop("genres")
df_title_crew=df_title_crew.drop("directors")
df_title_crew=df_title_crew.drop("writers")

#####dodajmy kolumny ideksów do nowych (jednokolumnowych) tabel i tabel pierwotnych

In [48]:
from pyspark.sql.functions import monotonically_increasing_id 

dodajmy kolumny indeksów do bazowych tabel

In [49]:
df_title_akas=df_title_akas.select("*").withColumn("id", monotonically_increasing_id())
df_name_basics=df_name_basics.select("*").withColumn("id", monotonically_increasing_id())
df_title_basics=df_title_basics.select("*").withColumn("id", monotonically_increasing_id())
df_title_crew= df_title_crew.select("*").withColumn("id", monotonically_increasing_id())

i do nowych, jednokolumnowych tabel

In [51]:
df_title_akas_types=df_title_akas_types.select("*").withColumn("id1", monotonically_increasing_id())
df_title_akas_attributes=df_title_akas_attributes.select("*").withColumn("id2", monotonically_increasing_id())
df_name_basics_primaryProfession=df_name_basics_primaryProfession.select("*").withColumn("id3", monotonically_increasing_id())
df_name_basics_knownForTitles=df_name_basics_knownForTitles.select("*").withColumn("id4", monotonically_increasing_id())
df_title_basics_genres=df_title_basics_genres.select("*").withColumn("id5", monotonically_increasing_id())
df_title_crew_directors=df_title_crew_directors.select("*").withColumn("id6", monotonically_increasing_id())
df_title_crew_writers=df_title_crew_writers.select("*").withColumn("id7", monotonically_increasing_id())

#####połączmy je odpowiednio na bazie numerów indeksu

In [55]:
df_title_akas= df_title_akas.join(df_title_akas_types, col('id') == col('id1'), 'leftouter')
df_title_akas= df_title_akas.join(df_title_akas_attributes, col('id') == col('id2'), 'leftouter')
df_name_basics= df_name_basics.join(df_name_basics_primaryProfession, col('id') == col('id3'), 'leftouter')
df_name_basics= df_name_basics.join(df_name_basics_knownForTitles, col('id') == col('id4'), 'leftouter')
df_title_basics=df_title_basics.join(df_title_basics_genres, col('id') == col('id5'), 'leftouter')
df_title_crew= df_title_crew.join(df_title_crew_directors, col('id') == col('id6'), 'leftouter')
df_title_crew= df_title_crew.join(df_title_crew_writers, col('id') == col('id7'), 'leftouter')

#####usuńmy kolumny indeksów

In [67]:
df_title_akas=df_title_akas.drop("id").drop("id1").drop("id2")
df_name_basics=df_name_basics.drop("id").drop("id3").drop("id4")
df_title_basics=df_title_basics.drop("id").drop("id5")
df_title_crew=df_title_crew.drop("id").drop("id6").drop("id7")

##Wyświetlmy dane

In [69]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basics.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+----------+----------------+---------+---------+--------------------+--------------------+
|    nconst|     primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+----------+----------------+---------+---------+--------------------+--------------------+
| nm0000001|    Fred Astaire|     1899|     1987|[soundtrack, acto...|[tt0053137, tt003...|
|nm11225060|  Dystinkt Beats|     null|     null|          [composer]|                [\N]|
| nm5925068|Ian Kosmaczewski|     null|     null|[camera_departmen...|[tt3132106, tt606...|
+----------+----------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+----------+--------+---------------+------+--------+---------------+-------------+----------+
|   titleId|ordering|          title|region|language|isOriginalTitle|        types|attributes|
+----------+--------+---------------+------+--------+---------------+-------------+----------+
| tt0000001|       1|     Карменсіта|    UA|  

Wyświetlmy typy danych

In [70]:
df_name_basics.printSchema()
df_title_akas.printSchema()
df_title_basics.printSchema()
df_title_crew.printSchema()
df_title_episode.printSchema()
df_title_principals.printSchema()
df_title_ratings.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- deathYear: integer (nullable = true)
 |-- primaryProfession: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- knownForTitles: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- isOriginalTitle: boolean (nullable = true)
 |-- types: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- attributes: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: boolean (nullable = true)
 